## SDSS Astroquery Batch

Batch runs through the images used in the GZ1 (SDSS DR7) dataset and returns half-light radii etc for data cuts

In [1]:
import numpy as np
import pandas as pd
import time
import math
import os
import random
import astropy.units as u
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord

In [2]:
CATALOG_PATH = '../Data/gz1_desi_cross_cat.csv'
OUTPUT_PATH = '../Data/gz1_desi_cross_cat_queried2.csv'
RADIUS = "1 arcsec"
catalog = pd.read_csv(CATALOG_PATH)

In [67]:
def split_dataframe(data, no_of_batches):
    batch_size = math.ceil(data.shape[0] / no_of_batches)
    batched_df = [data[i:i+batch_size] for i in range(0,data.shape[0], batch_size)]
    return batched_df

def get_SDSS_info_batch():
    batched_df = split_dataframe(catalog,2000) #30s per batch, more than this seems to fail

    if os.path.exists(OUTPUT_PATH):
        os.remove(OUTPUT_PATH)

    for i, batch in enumerate(batched_df):
        print(f"Processing batch {i}")
        
        coords = SkyCoord(batch["RA"],batch["DEC"],unit=(u.hourangle, u.deg))
        results = SDSS.query_region(coords,data_release=7,radius=RADIUS,photoobj_fields=["objID","ra","dec","err_r","petroR50_r","petroR50Err_r"]).to_pandas()
        #results = SDSS.query_region(coords,data_release=7,radius=RADIUS,photoobj_fields=["objID","ra","dec","err_r","petroR50_r","petroR50Err_r"],spectro=True).to_pandas()
        #print(results)

        #Clean up OBJID fields
        batch.loc[:,'OBJID'] = batch['OBJID'].astype(str).str.strip()
        results.loc[:,'objID'] = results['objID'].astype(str).str.strip()

        #print(f"Length of batch: {len(batch)}, with {len(pd.unique(batch['OBJID']))} unique")
        #print(f"Length of results: {len(results)}, with {len(pd.unique(results['objID']))} unique")
        #results.to_csv("test.csv")
        #batch.to_csv("test2.csv")

        
        i=0
        j=i
        rows_list = []
        while i < len(batch)-1: #Run through each item in batch
            #final_row = {}
            batch_row = batch.loc[i]
            results_row = results.loc[j]
            
            if batch_row['OBJID'] == results_row['objID']: #If OBJIDs match
                #print(f"Match at row {i}")
                if batch.loc[i+1,'OBJID'] == results.loc[j+1,'objID']: #If next object OBJIDs match
                    #print(f"Adding row {i} as next row matches")
                    batch_dict = batch_row.to_dict()
                    results_dict = results_row.to_dict()
                    batch_dict.update(results_dict)# Add matching rows from batch and results
                    rows_list.append(batch_dict)
                else:
                    #print(f"Skipping row {i} as next row does not match")
                    while batch.loc[i+1,'OBJID'] != results.loc[j+1,'objID']:
                        j += 1 # Move through results until match found
            i += 1 #Move on to next i
            j += 1 # Move on to next j

        final_columns = batch.columns.to_list()+results.columns.to_list()
        final = pd.DataFrame(rows_list,columns= final_columns)
        reduced = final.drop(["Unnamed: 0","objID","ra","dec"],axis=1)
        #print(f"Length of final: {len(final)}, with {len(pd.unique(final['OBJID']))} unique")
        # # Merge dataframes based on different id column names
        # merged_df = pd.merge(batch,results, left_on='OBJID', right_on='objID', how='inner',sort=False)
        # #print(merged_df.head(10))
        # print(f"Length of batch & results merged by OBJID: {len(merged_df)}, with {len(pd.unique(merged_df['OBJID']))} unique")

        # merged_df2 = pd.merge(batch, results, left_on='OBJID', right_on='objID', how='left', indicator=True)
        # #print(merged_df2.head(10))
        # rows_only_in_df2 = merged_df2[merged_df2['_merge'] != 'both']

        # # Get the indexes of rows in results that do not appear in batch
        # indexes_df2_not_in_df1 = rows_only_in_df2.index.tolist()

        # print("Indexes of rows in df2 that do not appear in df1:", indexes_df2_not_in_df1)
        # #print(merged_df.head(10))
        # # Get the indexes of rows in dataframe 2 that have a matching id to dataframe 1
        # matching_indexes_df2 = merged_df.index.tolist()

        # print(np.max(np.diff(matching_indexes_df2)))
        # print(matching_indexes_df2)


        # merged_batch = pd.merge(batch,results, how='inner', left_on='OBJID', right_on='objID') # this doesn't work

        # # Filter out extra objects
        # filtered_df = merged_batch[(merged_batch['OBJID'] != merged_batch['OBJID'].shift(-1))]

        # # Remove the last row if it's an extra object
        # if merged_batch.iloc[-1]['OBJID'] != merged_batch.iloc[-1]['OBJID']:
        #     filtered_df = filtered_df[:-1]


        time.sleep(1)
        #print(len(merged_batch))
        #print(len(filtered_df))


        #not_in_df1 = [objid for objid in results['objID'] if objid not in batch['OBJID']]

        #print(not_in_df1)
        #results.to_csv("test.csv")
        #batch.to_csv("test2.csv")
        #final.to_csv("test3.csv")
        reduced.to_csv(OUTPUT_PATH, mode='a', header=not os.path.exists(OUTPUT_PATH),index=False)

        break


get_SDSS_info_batch()

Processing batch 0


Length of batch: 324, with 324 unique
Length of results: 704, with 704 unique
Match at row 0
Adding row 0 as next row matches
Match at row 1
Adding row 1 as next row matches
Match at row 2
Skipping row 2 as next row does not match
Match at row 3
Skipping row 3 as next row does not match
Match at row 4
Skipping row 4 as next row does not match
Match at row 5
Skipping row 5 as next row does not match
Match at row 6
Adding row 6 as next row matches
Match at row 7
Adding row 7 as next row matches
Match at row 8
Skipping row 8 as next row does not match
Match at row 9
Adding row 9 as next row matches
Match at row 10
Adding row 10 as next row matches
Match at row 11
Skipping row 11 as next row does not match
Match at row 12
Skipping row 12 as next row does not match
Match at row 13
Skipping row 13 as next row does not match
Match at row 14
Adding row 14 as next row matches
Match at row 15
Adding row 15 as next row matches
Match at row 16
Adding row 16 as next row matches
Match at row 17
Addi

In [4]:
#reduced =  catalog.drop_duplicates(subset=['OBJID'])
reduced = catalog.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)
print(reduced.shape[0])
print(f"Number of galaxies in GZ1 catalogue: {catalog.shape[0]}")
print(f"Columns: {catalog.columns.values}")
reduced.head(10)
#reduced.to_csv(OUTPUT_PATH,index=False)

KeyError: "['Unnamed: 0.1'] not found in axis"